In [2]:
import tensorflow as tf
import numpy as np

In [45]:
sess = tf.InteractiveSession()
a = tf.convert_to_tensor([[40, 30, 20, 10], [10, 20, 30, 40]])
b = tf.nn.top_k(a, 2)

c = tf.gather_nd(a, b.indices[0])
# d = tf.gather_nd(a, b.indices)
print(a.shape)
print(sess.run(b))
print(sess.run(b.values))
print(sess.run(b.indices))
print(c.eval())
# print(d.eval())
sess.close()

(2, 4)
TopKV2(values=array([[40, 30],
       [40, 30]], dtype=int32), indices=array([[0, 1],
       [3, 2]], dtype=int32))
[[40 30]
 [40 30]]
[[0 1]
 [3 2]]
30


In [128]:
with tf.Session() as sess:
    batch, height, width, feature = 2, 3, 3, 2
    landmarks = 2*2
    style_images_num = 5
    style_n_best = 4
    
    x_landmarks = tf.placeholder(tf.float32, shape=(batch, landmarks))
    style_landmarks = tf.placeholder(tf.float32, shape=(style_images_num, landmarks))
    style_images = tf.placeholder(tf.float32, shape=(style_images_num, height, width, feature))
    
    init_x_landmarks = np.array(
        [
            [1,2,3,4],
            [9,8,7,6]
        ])
    
    init_style_landmarks = np.array(
        [
            [11, 12, 13, 14],
            [21, 22, 23, 24],
            [31, 32, 33, 34],
            [41, 42, 43, 44],
            [51, 52, 53, 54]
        ])
    
    init_style_images = np.array(
        [
            [[[201, 201], [202, 202], [203, 203]], [[204, 204], [205, 205], [206, 206]], [[207, 207], [208, 208], [209, 209]]],
            [[[301, 301], [302, 302], [303, 303]], [[304, 304], [305, 305], [306, 306]], [[307, 307], [308, 308], [309, 309]]],
            [[[401, 401], [402, 402], [403, 403]], [[404, 404], [405, 405], [406, 406]], [[407, 407], [408, 408], [409, 409]]],
            [[[501, 501], [502, 502], [503, 503]], [[504, 504], [505, 505], [506, 506]], [[507, 507], [508, 508], [509, 509]]],
            [[[601, 601], [602, 602], [603, 603]], [[604, 604], [605, 605], [606, 606]], [[607, 607], [608, 608], [609, 609]]]
        ])
    
    
    feed_dict = {
        x_landmarks: init_x_landmarks,
        style_landmarks: init_style_landmarks,
        style_images: init_style_images
    }
    
    x_tile = tf.tile(x_landmarks, [1, style_images_num])
    x_tile = tf.reshape(x_tile, [batch, style_images_num, landmarks])
    
    style_tile = tf.tile(style_landmarks, [batch, 1])
    style_tile = tf.reshape(style_tile, [batch, style_images_num, landmarks])
    
    mse = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(x_tile, style_tile)), 2))
    mse = tf.negative(mse) # 작은 값을 가져와야 하니깐!
    sort = tf.nn.top_k(mse, style_n_best)
    
    style_best = tf.gather(style_images, sort.indices)
    
    out = sess.run([style_best], feed_dict=feed_dict)

In [129]:
out

[array([[[[[ 201.,  201.],
           [ 202.,  202.],
           [ 203.,  203.]],
 
          [[ 204.,  204.],
           [ 205.,  205.],
           [ 206.,  206.]],
 
          [[ 207.,  207.],
           [ 208.,  208.],
           [ 209.,  209.]]],
 
 
         [[[ 301.,  301.],
           [ 302.,  302.],
           [ 303.,  303.]],
 
          [[ 304.,  304.],
           [ 305.,  305.],
           [ 306.,  306.]],
 
          [[ 307.,  307.],
           [ 308.,  308.],
           [ 309.,  309.]]],
 
 
         [[[ 401.,  401.],
           [ 402.,  402.],
           [ 403.,  403.]],
 
          [[ 404.,  404.],
           [ 405.,  405.],
           [ 406.,  406.]],
 
          [[ 407.,  407.],
           [ 408.,  408.],
           [ 409.,  409.]]],
 
 
         [[[ 501.,  501.],
           [ 502.,  502.],
           [ 503.,  503.]],
 
          [[ 504.,  504.],
           [ 505.,  505.],
           [ 506.,  506.]],
 
          [[ 507.,  507.],
           [ 508.,  508.],
           [ 

In [130]:
style_best

<tf.Tensor 'Gather_20:0' shape=(2, 4, 3, 3, 2) dtype=float32>

In [123]:
np.asarray(out).shape

(1, 2, 2, 3, 3, 2)

In [115]:
out.indices

AttributeError: 'list' object has no attribute 'indices'

In [109]:
np.asarray(out).shape

(1, 2, 2, 2)

In [ ]:
elems = (np.array([1, 2, 3]), np.array([-1, 1, -1]))
alternate = map_fn(lambda x: x[0] * x[1], elems, dtype=tf.int64)